https://towardsdatascience.com/dont-overfit-how-to-prevent-overfitting-in-your-deep-learning-models-63274e552323

In [21]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

In [22]:
x_train = pd.read_csv("60daysData.csv")
x_test = pd.read_csv("10daysTesting.csv")
y_test = x_test[["event"]]
y_train = x_train[["event"]]
del x_test["event"]
del x_train["event"]

In [23]:
x_train = x_train.values
y_train = y_train.values
x_test = x_test.values
y_test = y_test.values

In [24]:
#x_test = 1 - x_test
#y_test = 1 - y_test
y_train = y_train.transpose()[0]
y_test = y_test.transpose()[0]
x_test_orig = x_test
y_test_orig = y_test

In [25]:
x_train.shape

(23816, 1963)

In [33]:
windownbr = 12
windowsize = 300
x_test = x_test_orig[windowsize * windownbr:(windowsize*windownbr)+windowsize,]
y_test = y_test_orig[windowsize * windownbr:(windowsize*windownbr)+windowsize,]
x_test.shape

(300, 1963)

In [35]:
# https://www.kaggle.com/ryanholbrook/binary-classification
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(300, input_shape=(1963, ), activation='sigmoid'),
  tf.keras.layers.Dropout(0.4),

  tf.keras.layers.Dense(100, activation='sigmoid'),
  tf.keras.layers.Dropout(0.4),

  tf.keras.layers.Dense(50, activation='sigmoid'),
  tf.keras.layers.Dropout(0.4),

#  tf.keras.layers.Dense(20, activation='sigmoid'),
#  tf.keras.layers.Dropout(0.4),

#  tf.keras.layers.Dense(10, activation='sigmoid'),
#  tf.keras.layers.Dropout(0.4),

#  tf.keras.layers.Dense(5, activation='sigmoid'),
#  tf.keras.layers.Dropout(0.4),

#  tf.keras.layers.Dense(100, activation='tanh'),
#  tf.keras.layers.Dropout(0.2),
    
#  tf.keras.layers.Dense(50, activation='tanh'),
#  tf.keras.layers.Dense(50, activation='tanh'),
#  tf.keras.layers.Dense(100, activation='tanh'),
#  tf.keras.layers.Dense(150, activation='tanh'),
#  tf.keras.layers.Dense(500, activation='tanh'),
#  tf.keras.layers.Dense(150, activation='tanh'),
#  tf.keras.layers.Dense(100, activation='tanh'),
#  tf.keras.layers.Dense(50, activation='tanh'),
   
#  tf.keras.layers.Dense(5, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

model.compile(
              optimizer='adam',
#              optimizer='sgd',

#              loss='sparse_categorical_crossentropy',
              loss='binary_crossentropy',
#              loss='mse',

              metrics=['accuracy']    
#              metrics=['binary_accuracy']
)

#model.compile(optimizer='sgd',
#             loss='mse',
#             metrics=[tf.keras.metrics.TruePositives()])
#model.fit(x_train, y_train, epochs=200, shuffle=True, batch_size=300000)

model.fit(
        x_train, 
        y_train, 
        epochs=30, 
        shuffle=True,
        callbacks=[es_callback],
        validation_split = .1,
        )
model.evaluate(x_test, y_test)
PrintReport()

Train on 21434 samples, validate on 2382 samples
Epoch 1/30
21434/21434 [==============================] - 3s 154us/sample - loss: 0.1280 - accuracy: 0.9680 - val_loss: 0.0532 - val_accuracy: 0.9836
Epoch 2/30
21434/21434 [==============================] - 3s 127us/sample - loss: 0.0726 - accuracy: 0.9804 - val_loss: 0.0546 - val_accuracy: 0.9798
Epoch 3/30
21434/21434 [==============================] - 3s 127us/sample - loss: 0.0560 - accuracy: 0.9848 - val_loss: 0.0589 - val_accuracy: 0.9790
Epoch 4/30
21434/21434 [==============================] - 3s 130us/sample - loss: 0.0445 - accuracy: 0.9883 - val_loss: 0.0717 - val_accuracy: 0.9798
300/1 [========================================================================================================================================================================================================================================================================================================================================================

True positives:  4.0
False positives:  3.0
True negatives:  294.0
False negatives:  1.0
precision:  0.5714285714285714
recall:  0.8
f1:  0.6666666666666666


In [ ]:
PrintReport()

In [9]:
def PrintReport():
    y_pred = model.predict(x_test)
    np.set_printoptions(threshold=np.inf)
    y_pred = y_pred.transpose()[0]
    y_pred[y_pred > 0.1]=1
    y_pred[y_pred <= 0.1]=0
    total_positive = sum(y_pred)
    total_positive
    t1 = y_pred - y_test
    fp = t1
    fp[fp < 0] = 0
    fp = sum(fp)
    tp = total_positive - fp
    fn = t1
    fn[fn>=0]=0
    fn = 1 - fn



    diff = y_test - y_pred
    #print('diff: ',diff)
    # Correct is 0 
    # FP is -1 
    # FN is 1
    #print('Correctly classified: ', np.where(diff == 0)[0])
    #print('Incorrectly classified: ', np.where(diff != 0)[0])
    #print('False positives: ', np.where(diff == -1)[0])
    #print('False negatives: ', np.where(diff == 1)[0])
    fp = -1*sum(diff[diff == -1])
    fn = sum(diff[diff==1])
    tn = y_test.shape[0] - sum(y_test) - fn
    print('True positives: ', tp)
    print('False positives: ', fp )
    print('True negatives: ', tn)
    print('False negatives: ', fn)
    tn = y_test.shape[0] - tp - fp - fn
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall)/ (precision + recall)
    print('precision: ', precision)
    print('recall: ', recall)
    print('f1: ', f1)